In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import copy
import re
import os 

# Pull the data

In [ ]:
dd = pd.read_csv('DataDict.csv')
df = pd.read_csv('WDI_CSV_2024_06_28/WDICSV.csv')
country = pd.read_excel('WDI_Country.xlsx')

gdp = df[df['Indicator Code'].isin(['NY.GDP.MKTP.CD'])]
gdp = gdp.drop(['Country Code','Indicator Name','Indicator Code'],axis=1)
gdp = gdp.melt(id_vars= ["Country Name"],var_name="Year",value_name="gdp")

imp = df[df['Indicator Code'].isin(['NE.IMP.GNFS.CD'])]
imp = imp.drop(['Country Code','Indicator Name','Indicator Code'],axis=1)
imp = imp.melt(id_vars= ["Country Name"],var_name="Year",value_name="imp")

gdp_cn = df[df['Indicator Code'].isin(['NY.GDP.MKTP.CN'])]
gdp_cn = gdp_cn.drop(['Country Code','Indicator Name','Indicator Code'],axis=1)
gdp_cn = gdp_cn.melt(id_vars= ["Country Name"],var_name="Year",value_name="gdp_cn")

df = df[df['Indicator Code'].isin(dd['Code in Source'])]
vlist = []
years = []
for i in dd.index:
    v = dd.loc[i,'Variable']
    code = dd.loc[i,'Code in Source']
    f = dd.loc[i,'Formula']
    if dd.isnull().loc[i,'Decimal Places']:
        d = dd.loc[i,'Decimal Places']
    else:
        d = int(dd.loc[i,'Decimal Places'])
    dt = df[df['Indicator Code'].isin([code])]
    dt = dt.drop(['Country Code','Indicator Name','Indicator Code'],axis=1)
    dt = dt.melt(id_vars= ["Country Name"],var_name="Year",value_name="val")
    if dd.isnull().loc[i,'Formula']:
        dt = dt
    elif f == "%'NE.IMP.GNFS.CD'":
        dt = dt.merge(imp, on = ['Country Name', 'Year'], how = 'left')
        dt['cval'] = dt['val']/dt['imp']*100
        dt = dt.drop(['val'], axis = 1)
        dt = dt.rename(columns={'cval':'val'})
    elif f =="%'NY.GDP.MKTP.CN'":
        dt = dt.merge(gdp_cn, on = ['Country Name', 'Year'], how = 'left')
        dt['cval'] = dt['val']/dt['gdp_cn']*100
        dt = dt.drop(['val'], axis = 1)
        dt = dt.rename(columns={'cval':'val'})
    elif f == '/1000000000':
        dt.val = dt.val/1000000000
    elif f == '/1000000':
        dt.val = dt.val/1000000
    else:
        dt = dt.merge(gdp, on = ['Country Name', 'Year'], how = 'left')
        dt['cval'] = dt['val']/dt['gdp']*100
        dt = dt.drop(['val'], axis = 1)
        dt = dt.rename(columns={'cval':'val'})
    if dd.isnull().loc[i,'Decimal Places']:
        dt = dt
    else:
        dt.val = round(dt.val, d)

    n = 'Country Name'
    y = 'Year'
    first_last = dt.sort_values([n,y]).groupby(dt[n])['val'] \
            .agg(['first', 'last']).reset_index()
    first_last = first_last.rename(columns={"first": "Earliest", "last": "MostRecent"})
    dt.Year = dt.Year.apply(pd.to_numeric, errors='coerce')


    dt = dt.pivot_table(index = n,
                    columns= y,
                    values = 'val').reset_index()
    dt = dt.merge(first_last,
                on = n,
                how = 'left')
    dt = country.merge(dt,
                    on = n,
                    how = 'left')
    dt = dt.drop(columns= n)
    # dt.to_csv(f'IFs Import/{v}.csv', index = False)

# Get Years

In [ ]:
import more_itertools as mit
def find_ranges(iterable):
    """Yield range of consecutive numbers."""
    for group in mit.consecutive_groups(iterable):
        group = list(group)
        if len(group) == 1:
            yield group[0]
        else:
            yield group[0], group[-1]

def side_values(num_list):
    results_list = sorted(num_list)
    results = (str(results_list[0]) + "-" + str(results_list[-1]))
    return results

# dd = pd.read_excel('DataDict.xlsx', 
#                 #    sheet_name='All_files_Import'
#                    )
dd = pd.read_csv('DataDict.csv')

tblist = []
years = []
duration = []
realduration = []
for tb in dd.Table:
    v = tb.replace("Series","")
    #v = v.replace("Rev23","")
    # print(v)
    dt = pd.read_csv(f"IFs Import/{v}.csv")
    dt = dt.drop(columns=['Earliest','MostRecent'])
    dt = dt.melt(id_vars= ["Country","FIPS_CODE"],var_name="Year",value_name="val")
    dt.Year = dt.Year.apply(pd.to_numeric, errors='coerce')
    tblist.append(v)
    ymax = dt.Year.max()
    ymin = dt.Year.min()
    # years.append(str(ymin)+"-"+str(ymax))
    year1 = []
    for l in list(find_ranges(dt.Year.unique())):
        try:
            somelist = side_values(l)
            year1.append(somelist)
            # print(somelist)
        except:
            year1.append(str(l))
            # print(l)
    years.append(year1)
    duration.append(ymax-ymin+1)
    dt = dt.pivot(index=["Country","FIPS_CODE"],
                  columns='Year',
                  values='val'
                  )
    realduration.append(dt.shape[1])
df = pd.DataFrame({'Variable': tblist,
              'Years': years,
              'Duration': duration,
              "Realcolnum": realduration
             })
df.Years = df['Years'].str.join(', ')

df
df.to_excel('Years.xlsx', index = False)

# Import

In [ ]:
dd = pd.read_csv('DataDict.csv')
conn = sqlite3.connect('IFsDataImport.db')
cursor = conn.cursor()
#
for tb in dd.Table:
    v = tb.replace("Series","")
    #v = v.replace("Rev23","")
    print(v)
    dt = pd.read_csv(f"IFs Import/{v}.csv")
    sql_drop_table = f"DROP TABLE IF EXISTS [{tb}];"
    sql_create_table = f"CREATE TABLE [{tb}] (Country VARCHAR (255), FIPS_CODE VARCHAR (255), "
    for c in dt.columns[2:]:
        sql_create_table += f"[{c}] DOUBLE(53),"
    sql_create_table = sql_create_table[:-1] + ");"
    cursor.execute(sql_drop_table)
    cursor.execute(sql_create_table)
    dt.to_sql(name=f'{tb}', con=conn, if_exists="append", index=False)
    conn.commit()
#dd_update.to_sql(name=f'DataDict', con=conn, if_exists="replace", index=False)
conn.close()

# Blend in

In [ ]:
# ifshist_path_old = 
conn_wdi = sqlite3.connect("IFsDataImport.db")
cursor_wdi = conn_wdi.cursor()
dd_wdi = pd.read_sql_query("SELECT * FROM [DataDict]", conn_wdi)
##
conn_ifs_old =  sqlite3.connect(r"C:\Users\Public\IFs\DATA\IFsHistSeries.db")
for tb in dd_wdi["Table"]:
    df_ifs = pd.read_sql_query(f"SELECT * FROM [{tb}]", conn_ifs_old)
    df_wdi = pd.read_sql_query(f"SELECT * FROM [{tb}]",conn_wdi)
    df_ifs = df_ifs.drop(columns=["Earliest", "MostRecent"]).sort_values(by=["Country"]).reset_index(drop=True)
    df_wdi = df_wdi.drop(columns=["Earliest", "MostRecent"]).sort_values(by=["Country"]).reset_index(drop=True)
    df_wdi_fill = df_wdi.copy()
   # fill in historical values
    col_emp = []
    for c in df_wdi.columns[2:]:
        if df_wdi[c].dropna().empty:
            if c in df_ifs.columns and not df_ifs[c].empty:
                df_wdi_fill[c] = df_ifs[c]
            else:
                col_emp.append(c)
        else:
            break 
    # add empty year columns to drop from the latest time
    col_rev = list(df_wdi.columns)
    col_rev.reverse()
    for c in col_rev:
        if df_wdi[c].dropna().empty:
            col_emp.append(c)
        else:
            break
    # fill in missing countries
    country_emp = []
    for c in df_wdi.Country:
        if df_wdi[df_wdi.Country==c].dropna(axis=1).shape[1]==2:
            df_wdi_fill = df_wdi_fill.loc[df_wdi_fill.Country!= c]
            country_emp.append(c)
    df_wdi_fill = pd.concat([df_wdi_fill,df_ifs[df_ifs.Country.isin(country_emp)]])
    #
    df_wdi_fill = df_wdi_fill.drop(columns=col_emp)
    # Add most recent & earliest columns
    df_wdi_fill = df_wdi_fill.sort_values(by=["Country"]).reset_index(drop=True)
    df = df_wdi_fill.copy(True)
    for c in df.columns[2:]:
        if df[c].dtypes == "O":
            df[c] = df[c].astype(float)
    Ear=[]
    Rec=[]
    for i in range(df.shape[0]):
        line=df.iloc[i,2:]
        line.dropna(inplace=True)
        if not line.empty:
            Ear.append(line.values[0])
            Rec.append(line.values[-1])
        else:
            Ear.append(np.NaN)
            Rec.append(np.NaN)
    df["Earliest"]=Ear
    df["MostRecent"]=Rec
    sql_drop_table = f"DROP TABLE IF EXISTS [{tb}];"
    sql_create_table = f"CREATE TABLE [{tb}] (Country VARCHAR (255), FIPS_CODE VARCHAR (255), "       
    for c in df.columns[2:]:
        sql_create_table += f"[{c}] DOUBLE(53),"
    sql_create_table = sql_create_table[:-1] + ");"
    cursor_wdi.execute(sql_drop_table)
    cursor_wdi.execute(sql_create_table)
    df.to_sql(name=f'{tb}', con=conn_wdi, if_exists="append", index=False)
conn_wdi.commit()
conn_wdi.close()
conn_ifs_old.close()
